In [1]:
import pandas as pd
import numpy as np
import requests
import gmaps
import matplotlib.pyplot as plt
from config import gKey

gmaps.configure(api_key = gKey)

apiURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

cityWeatherPath = "../WeatherPy/output/cities.csv"

cityWeather_df = pd.read_csv(cityWeatherPath)



In [2]:
#humidity heatmap

humidity = cityWeather_df["Humidity:"]
coordinates = cityWeather_df[["Latitude:", "Longitude:"]]

gMap = gmaps.figure()
heatLayer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 5)

gMap.add_layer(heatLayer)

gMap

Figure(layout=FigureLayout(height='420px'))

In [3]:
#Narrow down cities

narrowedCityWeather_df = cityWeather_df.loc[(cityWeather_df["Max Temp:"].between(65, 75, inclusive = True) )
                                            & (cityWeather_df["Cloud Coverage:"].between(20, 30, inclusive = False))
                                            & (cityWeather_df["Wind:"] < 5)
                                           ].dropna()

narrowedCityWeather_df

,Unnamed: 0,City:,Latitude:,Longitude:,Max Temp:,Humidity:,Cloud Coverage:,Wind:,Country:,Date:
132,132,alta floresta,-9.8756,-56.0861,72.86,41,27,1.25,BR,1628393674
181,181,abaza,52.6550,90.0928,74.08,51,21,2.84,RU,1628393802
283,283,san carlos,9.6612,-68.5827,73.60,86,26,2.39,VE,1628393838
337,337,oeiras do para,-2.0031,-49.8544,74.26,89,21,3.09,BR,1628393856


In [4]:
#create hotel pins for map

hotel_df = narrowedCityWeather_df[["City:","Country:", "Latitude:", "Longitude:"]]

hotel_df["Hotel Name"] = "initialize"

params = {"radius" : 5000,
          "types" : "lodging",
          "key" : gKey
         }

for x, row in hotel_df.iterrows(): 
    latitude = row["Latitude:"]
    longitude = row["Longitude:"]
    params["location"] = f'{latitude},{longitude}'
    
    print("fetching lodging data")
    response = requests.get(apiURL, params = params).json()
    
    try:
        hotel_df.loc[x, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("broaden your horizons, no lodging available")
        continue

hotel_df

<ipython-input-4-ca5fe25df4d1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = "initialize"


fetching lodging data


C:\Users\Kristian\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


fetching lodging data
fetching lodging data
fetching lodging data


,City:,Country:,Latitude:,Longitude:,Hotel Name
132,alta floresta,BR,-9.8756,-56.0861,Floresta Amazonica Hotel
181,abaza,RU,52.6550,90.0928,Гостевой дом «Кедровый»
283,san carlos,VE,9.6612,-68.5827,Hotel Paternopoli
337,oeiras do para,BR,-2.0031,-49.8544,Araticu Soft Hotel


In [9]:
#template formatting is not a fan of colons apparently
hotel_df = hotel_df.rename(columns = {'City:': 'City', 'Country:' : 'Country'})

info_box_template = """ <dl>
                        <dt>Name</dt><dd>{Hotel Name}</dd>
                        <dt>City</dt><dd>{City:}</dd>
                        <dt>Country</dt><dd>{Country:}</dd>
                        </dl>
                    """

hotelData = [info_box_template.format(**row) for x, row in hotel_df.iterrows()]

coordinates = hotel_df[["Latitude:", "Longitude:"]]

marker_layer = gmaps.marker_layer(coordinates, info_box_content = hotelData)

gMap.add_layer(marker_layer)

gMap

Figure(layout=FigureLayout(height='420px'))